# Import required packages

In [1]:
from os import listdir
from pickle import load
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import gc

In [2]:
import keras.layers as L
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D
from keras.layers import UpSampling2D
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import RepeatVector
from keras.layers import Reshape
from keras.layers import concatenate
from keras.utils import plot_model

# reading multiple ab files into one object using a loop 

In [3]:
path = "../dataset/ab/ab/"
ab_list = listdir(path)
ab=[]
for i in range(0,1): #use len(ab_list) in place of 1 to load all files
    p = path + "/ab"+str(i+1)+".npy"
    if i==0:
        ab = np.load(p)
        print(ab.shape)
    else:
        ab= np.concatenate((ab,np.load(p)))
        print(ab.shape)
ab = tf.convert_to_tensor(ab)

(10000, 224, 224, 2)


Importing L channel

In [4]:
path_l = "../dataset/l/gray_scale.npy"
l = tf.convert_to_tensor(np.load(path_l))
l = tf.expand_dims(l, axis=3)

Limiting the training size to 500

In [5]:
l = l[:500,:,:,:]
ab = ab[:500,:,:,:]

concatenating the L and ab layers to create full pictures

In [54]:
full_img = tf.concat((l,ab),axis = 3)
full_img.shape

TensorShape([500, 224, 224, 3])

converting lab into rgb for effective downsampling

In [ ]:
full_img_rgb = []
for i in range(0,full_img.shape[0]):
     if i==0:
          full_img_rgb = cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_Lab2RGB)
          print(full_img_rgb.shape)
          full_img_rgb=np.expand_dims(full_img_rgb,0)
     else:
          full_img_rgb = np.append(full_img_rgb,
                                         np.expand_dims(cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_Lab2RGB),0),axis=0)
full_img_rgb = tf.convert_to_tensor(full_img_rgb)

(224, 224, 3)


Resizing the images to 128*128 p

img = full_img_rgb[2,:,:,:]
#img = cv2.cvtColor(np.array(img), cv2.COLOR_Lab2RGB)
plt.imshow(img)

In [55]:
full_img = tf.image.resize_with_pad(
                full_img_rgb,
                128,128,"bilinear"
                
)

freeing up memory

In [53]:
del full_img_rgb
gc.collect()

318

img = full_img[2,:,:,:]
#img = cv2.cvtColor(np.array(img), cv2.COLOR_Lab2RGB)
plt.imshow(np.array(img).astype('int'))

COnverting RGB back into Lab

In [63]:
full_img_lab = []
for i in range(0,full_img.shape[0]):
     if i==0:
          full_img_lab = cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_RGB2Lab)
          #print(full_img_lab.shape)
          full_img_lab=np.expand_dims(full_img_lab,0)
     else:
          full_img_lab = np.append(full_img_lab,
                                         np.expand_dims(cv2.cvtColor(np.array(full_img[i,:,:,:]),cv2.COLOR_RGB2Lab),0),axis=0)
full_img = tf.convert_to_tensor(full_img_lab)

In [64]:
del full_img_lab, ab, l
gc.collect()

1491

# train and test split 70:30

In [20]:
train = full_img[:350,:,:,:]
test = full_img[350:,:,:,:]

Normalising the picture data

In [17]:
train_datagen = ImageDataGenerator(rescale = 1./255)
datagen = ImageDataGenerator(featurewise_center=True,
                            featurewise_std_normalization=True)
datagen.fit(test)

In [18]:
train_iterator = datagen.flow(train, batch_size=64)
test_iterator = datagen.flow(test, batch_size=64)
print('Batches train=%d, test=%d' % (len(train_iterator),
                                     len(test_iterator)))

Batches train=55, test=24


In [20]:
gray_train = train[:,:,:,1]
gray_test = test[:,:,:,1]

In [21]:
def UNet():
    Xinpt = L.Input([None, None, 1])
    X0 = L.Conv2D(64, (3, 3), padding='same')(Xinpt)
    X0 = L.BatchNormalization()(X0)
    X0 = L.LeakyReLU(alpha=0.2)(X0)    #l,b,64
    X0 = L.Conv2D(64, (3, 3), strides=1, padding='same')(X0)
    X0 = L.BatchNormalization()(X0)
    X0 = L.LeakyReLU(alpha=0.2)(X0)    #l,b,64
    
    X1 = L.MaxPool2D((2, 2), strides=2)(X0)    #l/2,b/2,64
    X1 = L.Conv2D(128, (3, 3), padding='same')(X1)
    X1 = L.BatchNormalization()(X1)
    X1 = L.LeakyReLU(alpha=0.2)(X1)
    X1 = L.Conv2D(128, (3, 3), padding='same')(X1)
    X1 = L.BatchNormalization()(X1)
    X1 = L.LeakyReLU(alpha=0.2)(X1)    #l/2,b/2,128
    
    X2 = L.MaxPool2D((2, 2), strides=2)(X1)    #l/4,b/4,128
    X2 = L.Conv2D(256, (3, 3), padding='same')(X2)
    X2 = L.BatchNormalization()(X2)
    X2 = L.LeakyReLU(alpha=0.2)(X2)
    X2 = L.Conv2D(256, (3, 3), padding='same')(X2)
    X2 = L.BatchNormalization()(X2)
    X2 = L.LeakyReLU(alpha=0.2)(X2)    #l/4,b/4,256
    
    X3 = L.MaxPool2D((2, 2), strides=2)(X2)    #l/8,b/8,256
    X3 = L.Conv2D(512, (3, 3), padding='same')(X3)
    X3 = L.BatchNormalization()(X3)
    X3 = L.LeakyReLU(alpha=0.2)(X3)
    X3 = L.Conv2D(512, (3, 3), padding='same')(X3)
    X3 = L.BatchNormalization()(X3)
    X3 = L.LeakyReLU(alpha=0.2)(X3)    #l/8,b/8,512
    
    X4 = L.MaxPool2D((2, 2), strides=2)(X3)    #l/16,b/16,512
    X4 = L.Conv2D(1024, (3, 3), padding='same')(X4)
    X4 = L.BatchNormalization()(X4)
    X4 = L.LeakyReLU(alpha=0.2)(X4)
    X4 = L.Conv2D(1024, (3, 3), padding='same')(X4)
    X4 = L.BatchNormalization()(X4)
    X4 = L.LeakyReLU(alpha=0.2)(X4)    #l/16,b/16,1024
    
    X4 = L.Conv2DTranspose(512, (2, 2), strides=2)(X4)    #l/8,b/8,512
    X4 = L.Concatenate()([X3, X4])     #l/8,b/8,1024
    X4 = L.Conv2D(512, (3, 3), padding='same')(X4)
    X4 = L.BatchNormalization()(X4)
    X4 = L.Activation('relu')(X4)
    X4 = L.Conv2D(512, (3, 3), padding='same')(X4)
    X4 = L.BatchNormalization()(X4)
    X4 = L.Activation('relu')(X4)    #l/8,b/8,512
    
    X3 = L.Conv2DTranspose(256, (2, 2), strides=2)(X4)    #l/4,b.4,256
    X3 = L.Concatenate()([X2, X3])     #l/4,b/4,512
    X3 = L.Conv2D(256, (3, 3), padding='same')(X3)
    X3 = L.BatchNormalization()(X3)
    X3 = L.Activation('relu')(X3)
    X3 = L.Conv2D(256, (3, 3), padding='same')(X3)
    X3 = L.BatchNormalization()(X3)
    X3 = L.Activation('relu')(X3)    #l/4,b/4,256
    
    X2 = L.Conv2DTranspose(128, (2, 2), strides=2)(X3)    #l/2,b/2,128
    X2 = L.Concatenate()([X1, X2])     #l/2,b/2,256
    X2 = L.Conv2D(128, (3, 3), padding='same')(X2)
    X2 = L.BatchNormalization()(X2)
    X2 = L.Activation('relu')(X2)
    X2 = L.Conv2D(128, (3, 3), padding='same')(X2)
    X2 = L.BatchNormalization()(X2)
    X2 = L.Activation('relu')(X2)   #l/2,b/2,128
    
    X1 = L.Conv2DTranspose(64, (2, 2), strides=2)(X2)    #l,b,64
    X1 = L.Concatenate()([X0, X1])    #l,b,128
    X1 = L.Conv2D(64, (3, 3), padding='same')(X1)
    X1 = L.BatchNormalization()(X1)
    X1 = L.Activation('relu')(X1)
    X1 = L.Conv2D(64, (3, 3), padding='same')(X1)
    X1 = L.BatchNormalization()(X1)
    X1 = L.Activation('relu')(X1)    #l,b,64
    
    X0 = L.Conv2D(3, (1, 1), strides=1)(X1)     #l,b,3 
    model = Model(inputs=Xinpt, outputs=X0)
    return model

In [22]:
model = UNet()
model.compile('adam', loss='mean_squared_error', metrics=['mae', 'acc'])

In [ ]:
from IPython import display
for epoch in range(1, 20):
    display.clear_output(wait=True)
    print('Epoch:', epoch)
    model.fit(gray_train, train, batch_size=64, validation_data=(gray_test, test))
    sample_images(test, gray_test, 5)
    model.save('unet.hdf5')

Epoch: 1
